# IDS Challenge 
# Teilprojekt 03: Direkte Roboter-Kinematik
## Ergebnisse 

### Gruppe: ___
### Tutor: ___

Teilprojekt 03 Roboter-Vorwärtskinematik

Dies ist Ihr Arbeitsbereich mit einigen bereitgestellten Funktionen. Lesen Sie die Aufgabenstellung und implementieren Sie alle Lösungen hier. Lesen Sie zuerst die `README.ipynb`.

In [ ]:
# Nützliche Bibliotheken und globale Parameter importieren
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
import math

# --- Global Parameters for the Robot and Environment ---
L1 = 6.0  # Length of the first robot arm link
L2 = 6.0  # Length of the second robot arm link
L3 = 1.0  # Length of the rope/link holding the gripper attachment point

# Robot Gripper Parameters
GRIPPER_RADIUS = 0.5 # Radius of the robot's gripper 


# Visualisierung


In [ ]:
def robot_draw(alpha, beta, active_stars_to_draw=[], unreachable_stars_to_draw=[]):
    
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_aspect('equal')
    

    ax.set_xlim(-5, 10) 
    ax.set_ylim(-5, 10) 
    
    # baseline
    baseline_x = np.array([-0.5, 12.0]) 
    baseline_y = np.array([0.0, 0.0])
    ax.plot(baseline_x, baseline_y, '-k')

    # Define colors
    NORMAL_LINK_COLOR = [0.2, 0.2, 0.8]
 

    # Initialize link and object colors to normal
    link1_color = NORMAL_LINK_COLOR
    link2_color = NORMAL_LINK_COLOR
    link3_color = NORMAL_LINK_COLOR
    object_color = [0.0, 0.0, 1.0]

    overall_collision_detected = False

    # robot points
    O = np.array([0.0, 0.0])
    P, Q, R, M, T_fk = robot_direct_kinematics(alpha, beta)


    ax.plot(np.array([O[0], P[0]]), np.array([O[1], P[1]]), color=link1_color)
    ax.plot(np.array([P[0], Q[0]]), np.array([P[1], Q[1]]), color=link2_color)
    ax.plot(np.array([Q[0], R[0]]), np.array([Q[1], R[1]]), color=link3_color)
    
    # circle (the object being carried)
    t = np.linspace(0.0, 2 * np.pi, 100)
    circle_x = M[0] + GRIPPER_RADIUS * np.cos(t)
    circle_y = M[1] + GRIPPER_RADIUS * np.sin(t)
    ax.fill_between(circle_x, circle_y, facecolor=object_color)
    ax.plot(circle_x, circle_y, '-k')

    
    return overall_collision_detected, M

## Direkte Kinematik

In der **direkten Kinematik** bestimmen wir die Positionen verschiedener Punkte unseres Roboterarms, insbesondere $P$, $Q$, $R$, $M$ und $T$, basierend auf den gegebenen Gelenkwinkeln $\alpha$ und $\beta$. Das bedeutet, wir berechnen die Position jedes Roboterteils, für den Fall, dass die Konfiguration seiner Gelenke bekannt ist. Beachten Sie, dass $x$ im Koordinatensystem **nach rechts** und $y$ **nach oben** zeigt. Positive Werte für $\alpha$ und $\beta$ bedeuten eine Rotation **gegen den Uhrzeigersinn**.

<!-- ![Robot Kinematics](2_link_robot.jpg) -->
<img src="2_link_robot.jpg" width="300">

Ziel ist es, die die Koordinaten der Punkte $P$, $Q$, $R$, $M$ und $T$ mathematisch in Abhängigkeit der beiden bekannten Winkel auszudrücken. Diese Formeln berücksichtigen auch die festen physikalischen Parameter des Roboters: die Längen der Armglieder $L_1$ und $L_2$, die Seillänge $L_3$ und den Radius $r$ des Objekts.

Angenommen, $O$ ist der Ursprung des Koordinatensystems. Mithilfe der Länge des ersten Glieds $L_1$ und des Winkels $\alpha$ erhalten wir die Position von $P$:

$$
P = \left[ 
\begin{array}{c}
x_P \\
y_P \\
\end{array}
\right] = 
\left[ 
\begin{array}{c}
x_O + L_1\cos{(\alpha)}\\
y_O + L_1\sin{(\alpha)}\\
\end{array}
\right]
=
\left[ 
\begin{array}{c}
L_1\cos{(\alpha)}\\
L_1\sin{(\alpha)}\\
\end{array}
\right]
$$


Ausgehend vom obigen Punkt $P$ können wir die Position von Punkt $Q$ mithilfe der Länge des zweiten Glieds $L_2$ und der Winkel $\alpha$ und $\beta$ berechnen:
$$
Q = \left[ 
\begin{array}{c}
x_Q \\
y_Q \\
\end{array}
\right] = 
\left[ 
\begin{array}{c}
x_P + L_2\cos{(\alpha + \beta)}\\
y_P + L_2\sin{(\alpha + \beta)}\\
\end{array}
\right]
$$

Nachdem die Positionen von $P$ und $Q$ bestimmt wurden, können wir die Positionen der **Punkte $R$, $M$ und $T$** direkt berechnen, indem wir die vertikalen Versätze aufgrund der Seillänge $L_3$ und des Objektradius $r$ berücksichtigen:

$$
R = \left[ 
\begin{array}{c}
x_R \\
y_R \\
\end{array}
\right] 
= 
\left[ 
\begin{array}{c}
x_Q \\
y_Q - L_3\\
\end{array}
\right],
\quad
M = \left[ 
\begin{array}{c}
x_M \\
y_M \\
\end{array}
\right] 
= 
\left[ 
\begin{array}{c}
x_R \\
y_R - r\\
\end{array}
\right],
\quad
T = \left[ 
\begin{array}{c}
x_T \\
y_T \\
\end{array}
\right] 
= 
\left[ 
\begin{array}{c}
x_M \\
y_M - r\\
\end{array}
\right]
$$

Schließlich wird die Position des Punktes $T$ (der als Werkzeugmittelpunkt (TCP) betrachtet wird) angegeben:
$$
T = \left[ 
\begin{array}{c}
x_T \\
y_T \\
\end{array}
\right] 
= 
\left[ 
\begin{array}{c}
L_1\cos{(\alpha)} + L_2\cos{(\alpha + \beta)} \\
L_2\sin{(\alpha)} + L_2\sin{(\alpha + \beta)} - L_3 - 2r\\
\end{array}
\right]
$$

### Aufgabe 1.1: Funktion für die direkte Kinematik definieren
Schreiben Sie basierend auf den obigen Gleichungen eine Funktion zur Berechnung der direkten Kinematik.

In [ ]:
def robot_direct_kinematics(alpha,beta):
    # Put your code here. TRY TO RETURN P Q R M T
    return 

### Überprüfung der direkten Kinematik

Führen Sie die Funktion `robot_draw()` mit unterschiedlichen Winkeln aus und schauen Sie, ob der Roboter sich korrekt verhält. Ein Beispiel mit zwei möglichen Winkeln ist bereits vorgegeben.

In [ ]:
# Use robot_draw function to visualize the robot
_, _ = robot_draw(np.pi/2, np.pi/8*5)

### Aufgabe 1.2: Arbeitsbereich des Roboters bestimmen
#### Definition des Arbeitsbereichs:
Führen Sie die untenstehende Funktion aus, um Ihren in der Folge generierten Arbeitsbereich visualisieren zu können.


In [ ]:
# Function to visualize the workspace
def plot_workspace(points):
    x = [p[0] for p in points]
    y = [p[1] for p in points]

    fig, ax = plt.subplots(figsize=(12,12))
    ax.scatter(x, y, c='green', s=20, marker='o', label="Points")
    _, _ = robot_draw(np.pi/2, np.pi/2)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.axis("equal")
    ax.set_xlim(-15, 15)
    ax.set_ylim(-15, 15)
    ax.grid(True)
    # ax.legend()
    plt.show()

#### Aufgabenbeschreibung:

Der Arbeitsbereich eines Roboters ist die Menge aller Punkte im Raum, die er erreichen kann. Betrachten wir erneut den einfachen, planaren Roboterarm mit zwei Gelenken mit den Sgement-Längen $L_1$ und $L_2$. Das erste Gelenk (die Roboterbasis) ist am Ursprung $(0,0)$ fixiert, und die beiden Drehgelenke $\alpha$ und $\beta$ verfügen über einen uneingeschränkten Bewegungsbereich (von $0$ bis $2\pi$). Kollisionen sind hierbei zu ignorieren.

Schreiben Sie zwei Funktionen:
##### 1) Visualisieren Sie den Arbeitsbereich mit der bereitgestellten Funktion ```plot_workspace```. **Tipps**: Erstellen Sie Sets mit allen möglichen Werten von $\alpha$ und $\beta$ und generieren Sie anschließend die Punkte mit ```robot_direct_kinematics```.

In [ ]:
T_list = []

# 1st Function to write
def  generate_workspace():
    # Your code here. TRY TO ADD POINTS INTO THE T_list
    return
generate_workspace()

# Visualize the workspace
plot_workspace(T_list)


##### 2) Prüfen Sie bei einem gegebenen Punkt in 2D, ob dieser innerhalb des Arbeitsbereichs liegt. **Tipps**: Leiten Sie anhand der mathematischen Formeln der direkten Kinematik die analytische Lösung her und schreiben Sie anschließend die Funktion. Notieren Sie Ihre analytische Lösung möglichst in einer Mardown-Zelle.

In [ ]:
# 2nd function to write
def check_point_in_workspace(point):
    # Your code here. TRY TO RETURN A BOOLEAN VALUE
    return 


In [ ]:
# Provided function to visualize the point with workspace
def plot_point_with_workspace(point):
    x = [p[0] for p in T_list]
    y = [p[1] for p in T_list]

    fig, ax = plt.subplots(figsize=(12,12))
    ax.scatter(x, y, c='red', s=10, marker='o', label="Points")
    ax.scatter(point[0], point[1], c='black', s=30, marker='o', label="Points")
    _, _ = robot_draw(np.pi/2, np.pi/2)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.axis("equal")
    ax.set_xlim(-15, 15)
    ax.set_ylim(-15, 15)
    ax.grid(True)
    plt.show()
    if check_point_in_workspace(point):
        print("The point is inside workspace")
    else:
        print("The point is NOT in the workspace")

In [ ]:
# put your point here to test
plot_point_with_workspace((0,10))